In [ ]:
# Loading en_core_web_md
!python -m spacy download en_core_web_md

In [2]:
# import spacy
import spacy
import pandas as pd

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
data=pd.read_csv('/content/drive/MyDrive/Team Blue NLP/train_cleaned.csv')
data.head()

,Unnamed: 0,id,comment_text,avg_count_caps,comment_cleaned,comment_tokenized,comment_cleaned_spell,comment_cleaned_no_stopwords,comment_cleaned_spell_no_stopwords,comment_cleaned_no_stopwords_lemm,comment_cleaned_spell_no_stopwords_lemm,comment_cleaned_lemm,comment_cleaned_spell_lemm,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,131272,be631fee6a996d52,List of Moroccan Dutch people \n\nPeople shoul...,0.054422,list of moroccan dutch people people should ...,"['List', 'of', 'Moroccan', 'Dutch', 'people', ...",list of moroccan dutch people people should ha...,list moroccan dutch people people wikipedia pa...,list moroccan dutch people people wikipedia pa...,list moroccan dutch people people wikipedia pa...,list moroccan dutch people people wikipedia pa...,list of moroccan dutch people people should ha...,list of moroccan dutch people people should ha...,0,0,0,0,0,0
1,55898,955b143f75755ac3,"Not much better, be careful should someone rep...",0.012658,not much better be careful should someone rep...,"['Not', 'much', 'better', 'be', 'careful', 'sh...",not much better be careful should someone repo...,much better careful someone report ban vandalism,much better careful someone report ban vandalism,much well careful someone report ban vandalism,much well careful someone report ban vandalism,not much good be careful should someone report...,not much good be careful should someone report...,0,0,0,0,0,0
2,86025,e623261850dd2aaa,I blocked the pair of you because you are not ...,0.022857,i blocked the pair of you because you are not ...,"['I', 'blocked', 'the', 'pair', 'of', 'you', '...",i blocked the pair of you because you are not ...,blocked pair write encyclopaedia plenty places...,blocked pair write encyclopaedia plenty places...,block pair write encyclopaedia plenty place pl...,block pair write encyclopaedia plenty place pl...,i block the pair of you because you be not her...,i block the pair of you because you be not her...,0,0,0,0,0,0
3,96930,068e0a8032a4cf5c,Thank You \n\nThanks for the link on the refer...,0.020833,thank you thanks for the link on the referen...,"['Thank', 'You', 'Thanks', 'for', 'the', 'link...",thank you thanks for the link on the reference...,thank thanks link reference desks crux orthodo...,thank thanks link reference desks crux orthodo...,thank thanks link reference desk crux orthodox...,thank thanks link reference desk crux orthodox...,thank you thank for the link on the reference ...,thank you thank for the link on the reference ...,0,0,0,0,0,0
4,82799,dd852a6da913a096,"I know of about 50,000 Bible Scholars who disa...",0.034146,i know of about _number_ bible scholars who di...,"['I', 'know', 'of', 'about', '_NUMBER_', 'Bibl...",i know of about _number_ bible scholars who di...,know _number_ bible scholars disagree yet libe...,know _number_ bible scholars disagree yet libe...,know _number_ bible scholar disagree yet liber...,know _number_ bible scholar disagree yet liber...,i know of about _number_ bible scholar who dis...,i know of about _number_ bible scholar who dis...,0,0,0,0,0,0


In [5]:
nlp = spacy.load('en_core_web_md', disable = ['ner', 'parser'])

In [14]:
import numpy as np
def process_text(text):
  """
  This function will use Spacy to perform stopword removal and lemmatization.
  """
  # doc = nlp(text)
  #processed_text = " ".join([token.lemma_ for token in doc if not token.is_stop])
  # this will get the word2vec embeddings for the processed text (the average of each token in the doc's word2vec embeddings)
  #return np.array(nlp(processed_text).vector)
  return np.array(nlp(text).vector)

In [15]:
data.comment_cleaned_no_stopwords = data.comment_cleaned_no_stopwords.astype(str)

In [16]:
data["vectors"] = data.comment_cleaned_no_stopwords.apply(process_text)

In [17]:
X = np.array([vector for vector in data["vectors"]])
y = data["toxic"]

In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)

In [19]:
print(f"Training dataset is {X_train.shape}")
print(f"Training target is {y_train.shape}")
print(f"Test dataset is {X_test.shape}")
print(f"Test target is {y_test.shape}")

Training dataset is (95742, 300)
Training target is (95742,)
Test dataset is (31914, 300)
Test target is (31914,)


In [20]:
from sklearn.linear_model import LogisticRegression
logistic_regression = LogisticRegression(max_iter=1000)
logistic_regression.fit(X_train, y_train)
training_predictions = logistic_regression.predict(X_train)
testing_predictions = logistic_regression.predict(X_test)
training_predictions[:5]

array([0, 0, 0, 1, 0])

In [38]:
y_prob = logistic_regression.predict_proba(X_test)[:,1]

In [22]:
from sklearn.metrics import confusion_matrix 
confusion_matrix(y_train, training_predictions)

array([[85534,  1086],
       [ 3918,  5204]])

In [23]:
confusion_matrix(y_test, testing_predictions)

array([[28436,   365],
       [ 1366,  1747]])

In [26]:
from sklearn.metrics import f1_score
f1_score(y_test, testing_predictions)

0.6687081339712918

In [28]:
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

recall_score(y_test, testing_predictions)

0.5611949887568262

In [29]:
precision_score(y_test, testing_predictions)

0.8271780303030303

In [30]:
from sklearn.metrics import roc_auc_score

In [39]:
roc_auc_score(y_test, y_prob)

0.9556514522101456